# Inizializzazione università e SCA

In [1]:
from SmartContractAuthority.sca import SmartContractAuthority
from University.university import University
from Student.student import Student
import json
from SimulationUtils.simulationUtils import *

SCA_PATH = './smart_contract_authority/'
UNIVERSITIES_PATH = './universities/'
STUDENTS_PATH = './students/'
CONTRACTS_BUILD_PATH = './SmartContracts/build'

## Creazione Smart Contract Authority
Viene associato un account eth alla Smart Contract Authority che fa il deploy dei propri contratti

In [ ]:
SmartContractAuthority.create_sca(SCA_PATH, CONTRACTS_BUILD_PATH)

### Creazione dell'istanza
Viene effettivamente caricata l'istanza della SCA a partire dal file JSON di persistenza generato nel blocco precedente

In [2]:
sca = SmartContractAuthority(SCA_PATH+"SCA", CONTRACTS_BUILD_PATH)

## Creazione Università
In questo blocco vengono create le università della rete assegnando loro un nome, che poi risulterà nelle directory dedicate, ed un account eth tramite il quale faranno il deploy dei contratti per la gestione di credenziali e studenti.

In [ ]:
University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

### Creazione dell'istanza
Viene effettivamente caricata l'istanza delle università a partire dal file JSON di persistenza generato nel blocco precedente

In [3]:
uni_origin = University(UNIVERSITIES_PATH+'University_Università Degli Studi Di Salerno', CONTRACTS_BUILD_PATH)

uni_erasmus = University(UNIVERSITIES_PATH+'University_UNICAEN', CONTRACTS_BUILD_PATH)

## Creazione studente

In [ ]:
Student.create_student(STUDENTS_PATH)

In [5]:
studente = Student(STUDENTS_PATH+'Student_AntonioCarbone')

Studente 'Antonio Carbone' caricato da ./students/Student_AntonioCarbone


## Accreditamento università create

In [ ]:
UID, sca_contract_address = sca.register_university(uni_origin)
uni_origin.update_uid(UID)
uni_origin.update_sca_contract_address(sca_contract_address)

UID, sca_contract_address = sca.register_university(uni_erasmus)
uni_erasmus.update_uid(UID)
uni_erasmus.update_sca_contract_address(sca_contract_address)

## Immatricolazione presso università di origine

In [ ]:
credenziale_studente,sid = uni_origin.register_student(studente)
studente.set_sid(sid)
studente.save_credential(credenziale_studente)

## Immatricolazione presso università estera

In [ ]:
uni_erasmus.register_erasmus_student(studente, studente.credentials[0].toJSON())
uni_erasmus.update_erasmus_students(studente, credenziale_studente.CID)

## Richiesta e rilascio credenziale carriera

In [ ]:
career_credential = uni_erasmus.request_career_credential(studente)
studente.save_credential(career_credential)

## Costruzione della credenziale parziale

In [6]:
partial_credential = studente.build_shared_credential()

studente.save_credential(partial_credential)


Seleziona una credenziale da condividere:
0: CID = CID:ENSICAEN:1
1: CID = CID:UNISA:1

Seleziona le proprietà da condividere (y/n):

Credenziale parziale costruita con successo:
{
    "certificateId": "CID:ENSICAEN:1",
    "studentId": "SID:UNISA:1",
    "universityId": "UID:ENSICAEN",
    "issuanceDate": "2023-10-01",
    "properties": [
        {
            "typology": "ErasmusInfo",
            "data": {
                "programName": "Erasmus+ Europe",
                "startActivity": "2024-09-01",
                "endActivity": "2025-01-31"
            },
            "nonce": "17529415642263201",
            "merkle_proof": [
                [
                    "4fbe29d15cce86212d155f4ee7199eb63cd84e7d376b4c68a712d191dd82d0af",
                    "R"
                ],
                [
                    "c26eecf947715aa1c48f749ac73ed87c4cf6beb5948dd2c4bbdbc31dbf22f7a7",
                    "R"
                ],
                [
                    "757541362764099f5b32c

## Condivisione credenziale

In [7]:
uni_origin.validate_shared_credential(studente,partial_credential.toJSON())

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.
Challenge di autenticazione superata.
Le property sono state verificate correttamente!
La credenziale condivisa è valida


## Revoca credenziale di immatricolazione

In [ ]:
uni_origin.revoke_cid("CID:UNISA:1")

## Revoca credenziale carriera

In [ ]:
uni_erasmus.revoke_cid("CID:ENSICAEN:0")

## Manomissione di una credenziale

In [9]:
parsed_partial_credential = json.loads(partial_credential.toJSON())
tampered_credential = tamper_all_credential_properties(parsed_partial_credential)
uni_origin.validate_shared_credential(studente, json.dumps(tampered_credential, ensure_ascii=False, indent=4))

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.
Challenge di autenticazione superata.


Exception: Errore durante la verifica della firma dell'issuer: Inconsistenza rilevata nella merkle proof della property 1 (Tipo: ExtraActivity). Root calcolata: f62297c187d25c9a57a0c5f40ab42a2a0c5a9c76ef70c60be7ddfd4e97bc60eb, Root attesa: f4719560480041abeafa78e8cf9e35f6222bbf0b89f09afd8be0502a5ab13532

## Ente certificatore fasullo

## Ente certificatore malintenzionato

## Sosia

## Forgiatore di credenziali